In [1]:
from typing import List, Tuple, Set, Dict, Optional, Callable
from copy import deepcopy

# Type alias for predicates
Predicate = Tuple[str, ...] # e.g. ("on", "A", "B"), ("clear", "A"), ("handempty",)

# Predicate helper functions
def pred_on(x: str, y: str) -> Predicate:
    return ("on", x, y)
def pred_clear(x: str) -> Predicate:
    return ("clear", x)
def pred_holding(x: str) -> Predicate:
    return ("holding", x)
def pred_handempty() -> Predicate:
    return ("handempty",)

# Operator representation
class Operator:
    """Represents a Blocks World operator (action)."""
    def __init__(self, name: str, args: Tuple[str, ...],
                 preconds: List[Predicate],
                 add_effects: List[Predicate],
                 del_effects: List[Predicate]):
        self.name = name
        self.args = args
        self.preconds = preconds
        self.add_effects = add_effects
        self.del_effects = del_effects

    def __repr__(self):
        if self.args:
            return f"{self.name}({', '.join(self.args)})"
        return f"{self.name}()"

    def apply(self, state: Set[Predicate]):
        """Modifies the state by applying delete and add effects."""
        # Apply delete effects.
        for d in self.del_effects:
            if d in state:
                state.remove(d)
        # Apply add effects
        for a in self.add_effects:
            state.add(a)

# Ground operator factories
def make_Pickup(x: str) -> Operator:
    """Pickup block x from the table."""
    pre = [pred_clear(x), pred_on(x, "table"), pred_handempty()]
    adds = [pred_holding(x)]
    dels = [pred_clear(x), pred_on(x, "table"), pred_handempty()]
    return Operator("Pickup", (x,), pre, adds, dels)

def make_Putdown(x: str) -> Operator:
    """Putdown block x onto the table."""
    pre = [pred_holding(x)]
    adds = [pred_on(x, "table"), pred_clear(x), pred_handempty()]
    dels = [pred_holding(x)]
    return Operator("Putdown", (x,), pre, adds, dels)

def make_Unstack(x: str, y: str) -> Operator:
    """Unstack block x from block y."""
    pre = [pred_on(x, y), pred_clear(x), pred_handempty()]
    adds = [pred_holding(x), pred_clear(y)]
    dels = [pred_on(x, y), pred_clear(x), pred_handempty()]
    return Operator("Unstack", (x, y), pre, adds, dels)

def make_Stack(x: str, y: str) -> Operator:
    """Stack block x onto block y."""
    pre = [pred_holding(x), pred_clear(y)]
    adds = [pred_on(x, y), pred_clear(x), pred_handempty()]
    dels = [pred_holding(x), pred_clear(y)]
    return Operator("Stack", (x, y), pre, adds, dels)

# Goal-stack planner
class GoalStackPlanner:
    """Implements the Goal-Stack Planning algorithm."""
    def __init__(self, initial_state: Set[Predicate], goal_state: Set[Predicate], blocks: List[str]):
        self.state = deepcopy(initial_state) # current world state
        self.goal = deepcopy(goal_state)     # set of desired predicates
        self.blocks = blocks                 # list of block names
        self.plan: List[Operator] = []       # resulting plan
        # stack elements are either Predicate (goal) or Operator instance (to be applied)
        self.stack: List[Operator | Predicate] = []

    def is_satisfied(self, predicate: Predicate) -> bool:
        """Checks if a predicate is true in the current state."""
        return predicate in self.state

    def select_operator_for(self, goal: Predicate) -> Optional[Operator]:
        """Chooses an operator that satisfies the given goal predicate."""
        pred_name = goal[0]

        if pred_name == "on":
            x, y = goal[1], goal[2]
            if y == "table":
                # Goal: on(x, table) => use Putdown(x)
                return make_Putdown(x)
            else:
                # Goal: on(x, y) where y is a block => use Stack(x, y)
                return make_Stack(x, y)

        if pred_name == "holding":
            x = goal[1]
            # Goal: holding(x). Check where x is currently.
            # Prefer Unstack if on another block, otherwise Pickup if on table.
            for p in self.state:
                if p[0] == "on" and p[1] == x:
                    z = p[2]
                    if z == "table":
                        return make_Pickup(x)
                    else:
                        return make_Unstack(x, z)
            # If not 'on' anything (shouldn't happen in valid state), fall back to Pickup
            return make_Pickup(x) 

        if pred_name == "clear":
            x = goal[1]
            # Goal: clear(x). Achieved by Unstack(w, x) if some block w is on x.
            for p in self.state:
                if p[0] == "on" and p[2] == x:
                    w = p[1] # w is the block on x
                    return make_Unstack(w, x)
            # Already clear, no operator needed
            return None

        if pred_name == "handempty":
            # Goal: handempty(). Achieved by Putdown(x) if holding(x).
            for p in self.state:
                if p[0] == "holding":
                    x = p[1]
                    return make_Putdown(x)
            # Already handempty, no operator needed
            return None
        
        return None

    def goal_stack_planning(self) -> Optional[List[Operator]]:
        """Main planning loop."""
        # initialize stack with all goal predicates (reversed sort for deterministic order)
        for g in sorted(self.goal, key=str, reverse=True):
            self.stack.append(g)

        visited_iterations = 0
        max_iterations = 10000 

        while self.stack:
            if visited_iterations > max_iterations:
                print("Exceeded max iterations. Planning failed.")
                return None
            visited_iterations += 1
            
            top = self.stack.pop()

            # If top is a predicate (goal)
            if isinstance(top, tuple):
                top_pred: Predicate = top
                # Goal already satisfied?
                if self.is_satisfied(top_pred):
                    continue
                
                # Choose operator to satisfy it
                op = self.select_operator_for(top_pred)
                if op is None:
                    print(f"No operator found to achieve goal {top_pred}; planning failed.")
                    return None
                
                # Push operator (as a marker) and then its preconditions
                self.stack.append(op)
                
                # Push preconditions that are not currently satisfied
                # Reversed so first precond is processed first
                for prec in reversed(op.preconds): 
                    if not self.is_satisfied(prec):
                        self.stack.append(prec)

            # Top is an operator instance
            else:
                op: Operator = top
                
                # Check if all preconditions are satisfied
                unsatisfied = [p for p in op.preconds if not self.is_satisfied(p)]
                
                if not unsatisfied:
                    # Apply operator
                    op.apply(self.state)
                    self.plan.append(op)
                else:
                    # Operator not yet applicable; push it back and push unsatisfied preconditions
                    self.stack.append(op)
                    for prec in reversed(unsatisfied):
                        self.stack.append(prec)

        return self.plan

# Utilities to build states
def make_state(on_list: List[Tuple[str, str]]) -> Set[Predicate]:
    """
    Builds a Blocks World state (set of predicates) from a list of (block, location) tuples.
    Also computes 'clear' and 'handempty' predicates initially.
    """
    state: Set[Predicate] = set()
    
    # 1. Add all 'on' predicates
    for x, y in on_list:
        state.add(pred_on(x, y))
        
    # 2. Compute 'clear' predicates
    # Identify all blocks (excluding 'table')
    blocks = {x for x, _ in on_list} | {y for _, y in on_list if y != "table"}
    
    for b in blocks:
        # Check if any block w is on block b
        occupied = any(p for p in state if p[0] == "on" and p[2] == b)
        if not occupied:
            state.add(pred_clear(b))
            
    # 3. Add 'handempty' (assuming initial state is always handempty)
    state.add(pred_handempty())
    return state

def print_plan(plan: List[Operator]):
    """Prints the final plan (sequence of operators)."""
    if not plan:
        print("No plan (empty or failed).")
        return
    print("Plan ({} steps): ".format(len(plan)))
    for i, op in enumerate(plan, 1):
        print(f"{i}. {op}")

# Examples
if __name__ == "__main__":
    # Example 1: Simple make A on B given A on table, B on table
    blocks1 = ["A", "B", "C"]
    initial_on1 = [("A", "table"), ("B", "table"), ("C", "table")]
    initial_state1 = make_state(initial_on1)
    goal_preds1 = {pred_on("A", "B")} # goal: on(A, B)
    
    planner1 = GoalStackPlanner(initial_state1, goal_preds1, blocks1)
    plan1 = planner1.goal_stack_planning()
    
    print("Example 1:")
    print_plan(plan1)
    print()

    # Example 2: More classical: initial: A on B, B on table, C on table
    # goal: on(C, A) and on(A, B) 
    blocks2 = ["A", "B", "C"]
    initial_on2 = [("A", "B"), ("B", "table"), ("C", "table")]
    initial_state2 = make_state(initial_on2)
    goal_preds2 = {pred_on("C", "A"), pred_on("A", "B")} # both goals
    
    planner2 = GoalStackPlanner(initial_state2, goal_preds2, blocks2)
    plan2 = planner2.goal_stack_planning()
    
    print("Example 2:")
    print_plan(plan2)
    print()

Example 1:
Plan (2 steps): 
1. Pickup(A)
2. Stack(A, B)

Example 2:
Plan (2 steps): 
1. Pickup(C)
2. Stack(C, A)

